## 💡 lite helpers

jupyterlite is _magic_, but sometimes needs... _more magic_.

In [ ]:
from __future__ import annotations

import base64
import re
import tarfile
import tempfile
import time
from pathlib import Path

from IPython.display import IFrame, Markdown, display

## data uris

the `data:` uri scheme lets us inject just about any standalone content into an `iframe`. using `base64` encoding is heavier, but works for text and binary data.

In [ ]:
def data_uri(data: bytes, mime: str) -> str:
    data_b64 = base64.b64encode(data).decode("utf-8")
    return f"data:{mime};base64,{data_b64}"

## data uri iframes

the `src` of an `iframe` can be a data uri.

In [ ]:
def data_uri_iframe(data: bytes, width, height):
    display(IFrame(data_uri(data, "text/html"), width=width, height=height))

## inlining assets


In [ ]:
def asset_uris(root_path: Path, globs: dict[str, str] | None = None) -> dict[str, str]:
    assets = {}
    globs = globs or {"*.js": "application/json", "*.css": "text/css", "*.png": "image/png"}
    for glob, mime in globs.items():
        for asset in root_path.rglob(glob):
            assets[str(asset.relative_to(root_path).as_posix())] = data_uri(
                asset.read_bytes(), mime
            )
    return assets

## special cases

### pytest html
`pytest-html` uses some brower APIs not available in `iframe` elements with `src="data:..."`. This wrapper cleans them up enough to _work_, but not all features will be enabled.

In [ ]:
def pytest_html(path, width="100%", height="600px"):
    text = Path(path).read_text(encoding="utf-8")
    text = text.replace("sessionStorage", "fakeSessionStorage").replace(
        "<script>",
        """<script>
            class FakeSessionStorage {
                constructor() { this._data = {}; }
                getItem = (key) => this._data[key] || null;
                setItem = (key, value) => this._data[key] = value;
            }
            window.fakeSessionStorage = new FakeSessionStorage();
        """,
    )
    data_uri_iframe(text.encode(), width, height)

### coverage html

we can rewrite links to web assets with inline URIs, and then re-encode them as `iframe` elements.

In [ ]:
def coverage_html(root, width="100%", height="600px"):
    root_path = Path(root)
    assets = asset_uris(root_path)

    for path in sorted(root_path.glob("*.html")):
        text = path.read_text(encoding="utf-8")
        for filename, uri in assets.items():
            rel = (root_path / filename).relative_to(path.parent, walk_up=True)
            for attr in ["href", "src"]:
                text = re.sub(rf'{attr}="{filename}([?#][^"]*)?"', f'{attr}="{uri}"', text)
        data_uri_iframe(text.encode(), width, height)

### sphinx html

In [ ]:
def sphinx_html(root, width="100%", height="600px"):
    coverage_html(root, width, height)

## saving the demo

jupyterlite doesn't offer a simple way to download the current state of all the files.

In [ ]:
def demo_tarball(root: str = "/drive", out_name: str = "importnb-demo-{now}.tgz") -> None:
    root_path = Path(root)
    md = [
        "| file | status | note |",
        "|------|:------:|------|",
    ]
    with tempfile.TemporaryDirectory() as tmp:
        tmp_path = Path(f"{tmp}/{out_name}".format(now=int(time.time())))
        with tarfile.open(f"{tmp_path}", "w|gz") as tf:
            for path in sorted(root_path.rglob("*.*")):
                arcname = f"""{path.relative_to(root_path)}"""
                line = [f"`{arcname}`", "", ""]
                if path.name.endswith(".tgz"):
                    line[1], line[2] = ["skip", "extension"]
                    continue
                try:
                    tf.add(arcname, arcname)
                    line[1] = "ok"
                except OSError as err:
                    line[1], line[2] = ["error", f"{err}"]
                md += [" | ".join(line)]
        uri = data_uri(tmp_path.read_bytes(), "application/tar+gzip")
        md = [
            f"""<h3><a href="{uri}" download="{tmp_path.name}">
            download <code>📁{tmp_path.name}</code> ({int(len(uri) / 1024)}kb)
        </a></h3>""",
            "",
            *md,
        ]
        display(Markdown("\n".join(md)))


if __name__ == "__main__":
    demo_tarball()